<div style="text-align: center;">
    <h1><strong>Evaluate a Resnet Model </strong></h1>
    <h1><strong>Sqlite & Local Folder</strong></h1>
</div> 

# Goal of the notebook:
#### The purpose of the notebook is to load a Resnet model previously trained and to evaluate its performance  on a test dataset
#### We assumed that the test dataset is generated according to the same approach than the one used to train the model (see train_resnet_model_local.ipynb or train_resnet_model_local.ipynb)
#### Albeit images used for testing should not have been used during training for data leakage consideration (ie. no date overlap)
#### Evaluation metrics displayed here are the ones that have been set while compiling the model before training (Loss, Accuracy, Precision and Recall)
#### The notebook is meant to be used if the storage solution is set for <strong>Sqlite & Local Folder</strong>

# Summary:
### 1- Import of Packages and Dependencies
### 2- Import Environment Variables
### 3- Finding MLFlow experiments and runs 
### 4- Generate the test dataset
### 5- Evaluate the model on the test dataset

# 1- Import of Packages and Dependencies

In [1]:
import os
from dotenv import load_dotenv
from utils.build_dataset import *
import mlflow

2025-01-03 08:22:08.067939: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-03 08:22:08.068344: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-03 08:22:08.070454: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-03 08:22:08.076293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735888928.085544   10260 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735888928.08

# 2- Import Environment Variables

In [2]:
# Load environment variables from the .env file
load_dotenv()

# Access environment variables using os.getenv() method
# We need api_key and pai_url to connect to the API and get the data
api_key = os.getenv("API_KEY")
api_url = os.getenv("API_URL")

# 3- Finding MLFlow experiments and runs  

#### We first need to define the tracking URI for MLflow so that it can log the results

In [3]:
# We first need to define the tracking URI for MLflow so that it can log the results
tracking_uri=f"sqlite:///mlflow.db"
mlflow.set_tracking_uri(tracking_uri)

#### Now we can list our experiments and select the one we want to use

In [4]:
# List all experiments
experiments = mlflow.search_experiments()

# Print the experiment details
for experiment in experiments:
    print(f"Experiment ID: {experiment.experiment_id}")
    print(f"Name: {experiment.name}")
    print(f"Artifact Location: {experiment.artifact_location}")
    print(f"Lifecycle Stage: {experiment.lifecycle_stage}")
    print("------------------------------")

Experiment ID: 2
Name: test
Artifact Location: /home/jerome/code/J-Pouzoulet/training_resnet_v2/training_resnet/mlruns/2
Lifecycle Stage: active
------------------------------
Experiment ID: 1
Name: my_experiment_v22
Artifact Location: /home/jerome/code/J-Pouzoulet/training_resnet_v2/training_resnet/mlruns
Lifecycle Stage: active
------------------------------
Experiment ID: 0
Name: Default
Artifact Location: /home/jerome/code/J-Pouzoulet/training_resnet_v2/training_resnet/mlruns/0
Lifecycle Stage: active
------------------------------


#### Now we choose the ID of the experiment we want to log the results to

In [8]:
# we set the ID of the experiment we want to use
experiment_id = "2"
# We search for the runs in the experiment
runs = mlflow.search_runs(experiment_ids=experiment_id)
# We view the runs in the experiment (a pd.DataFrame)
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.val_recall,metrics.val_loss,metrics.accuracy,metrics.recall,...,tags.mlflow.source.type,tags.mlflow.gitRepoURL,tags.mlflow.project.backend,tags.mlflow.user,tags.mlflow.project.env,tags.mlflow.project.entryPoint,tags.mlflow.runName,tags.mlflow.log-model.history,tags.mlflow.source.git.repoURL,tags.mlflow.source.git.commit
0,49ecfd90cba74fc9801a6d7a89a85d09,2,FAILED,/home/jerome/code/J-Pouzoulet/training_resnet_...,2025-01-02 21:47:30.716000+00:00,2025-01-02 21:48:37.629000+00:00,1.0,0.000024,1.0,1.0,...,PROJECT,git@github.com:J-Pouzoulet/training_resnet.git,local,jerome,virtualenv,main,merciful-squid-306,"[{""run_id"": ""49ecfd90cba74fc9801a6d7a89a85d09""...",git@github.com:J-Pouzoulet/training_resnet.git,933e8320f7dc4e6fec468cd9436350df003bcf7a
1,84b0ec45ef6841b0b0c385171e011702,2,FAILED,/home/jerome/code/J-Pouzoulet/training_resnet_...,2025-01-02 21:44:51.608000+00:00,2025-01-02 21:46:01.348000+00:00,1.0,0.000009,1.0,1.0,...,PROJECT,git@github.com:J-Pouzoulet/training_resnet.git,local,jerome,virtualenv,main,chill-hare-491,"[{""run_id"": ""84b0ec45ef6841b0b0c385171e011702""...",git@github.com:J-Pouzoulet/training_resnet.git,933e8320f7dc4e6fec468cd9436350df003bcf7a
2,5f8c32e982a44593bcde9f722d86acaa,2,FAILED,/home/jerome/code/J-Pouzoulet/training_resnet_...,2025-01-02 21:39:07.899000+00:00,2025-01-02 21:40:13.172000+00:00,1.0,0.000007,1.0,1.0,...,PROJECT,git@github.com:J-Pouzoulet/training_resnet.git,local,jerome,virtualenv,main,bouncy-shrike-608,"[{""run_id"": ""5f8c32e982a44593bcde9f722d86acaa""...",git@github.com:J-Pouzoulet/training_resnet.git,933e8320f7dc4e6fec468cd9436350df003bcf7a


In [9]:
runs = mlflow.search_runs(experiment_ids=experiment_id)

# Print the experiment details
for index in runs.index:
    print(f"run ID: {runs[['run_id']].iloc[index]}")
    print(f"Mlflow name: {runs[['tags.mlflow.runName']].iloc[index]}")
    print(f"Artifact URI: {runs[['artifact_uri']].iloc[index]}")
    print("------------------------------")

run ID: run_id    49ecfd90cba74fc9801a6d7a89a85d09
Name: 0, dtype: object
Mlflow name: tags.mlflow.runName    merciful-squid-306
Name: 0, dtype: object
Artifact URI: artifact_uri    /home/jerome/code/J-Pouzoulet/training_resnet_...
Name: 0, dtype: object
------------------------------
run ID: run_id    84b0ec45ef6841b0b0c385171e011702
Name: 1, dtype: object
Mlflow name: tags.mlflow.runName    chill-hare-491
Name: 1, dtype: object
Artifact URI: artifact_uri    /home/jerome/code/J-Pouzoulet/training_resnet_...
Name: 1, dtype: object
------------------------------
run ID: run_id    5f8c32e982a44593bcde9f722d86acaa
Name: 2, dtype: object
Mlflow name: tags.mlflow.runName    bouncy-shrike-608
Name: 2, dtype: object
Artifact URI: artifact_uri    /home/jerome/code/J-Pouzoulet/training_resnet_...
Name: 2, dtype: object
------------------------------


#### We will use here the first run of the experiment as an example (e.g. at the index 0).
#### We need the run_id to load the model.

In [10]:
run_id = mlflow.search_runs(experiment_ids=experiment_id).iloc[0].run_id

In [11]:
print(f"The run ID is : {run_id}")

The run ID is : 49ecfd90cba74fc9801a6d7a89a85d09


#### Now we can load the model from the run.

In [13]:
# Load the model from the MLflow run
model_uri = f"runs:/{run_id}/model"  # replace with the actual run ID

# Load the Keras model
loaded_model = mlflow.keras.load_model(model_uri)

# 4- Generate the test dataset

#### We first need to get the name of the Resnet model from which our model was fine-tuned to propress the test dataset
#### We can get this information from the MLflow run metadata. 

In [14]:
# from MLFlow we call the parameters of the model and get the model_name (to be passed in the propressing function)
params = mlflow.get_run(run_id).data.params
model_name = params['model_name']
print(f"Model name: {model_name}")

Model name: ResNet50


#### We set the parameters to built the test dataset like we did for the training dataset
#### We use a slighty different function to create the test dataset because we don't need to split the data
#### Otherwise the pipeline is the same

In [15]:
image_dir = "media"
labels = ["vine", "grass", "ground"]
start_date = "2021-03-01"
end_date = "2021-05-01"

# We make the train and validation datasets
image_urls = get_image_urls_with_multiple_labels(labels, start_date, end_date, api_key, api_url)
# Download images and create a sample map
df_sample_map = create_sample_map(image_urls)
df_sample_map = download_images(df_sample_map, image_dir)

test_dataset = create_test_dataset(df_sample_map,
                              image_dir = 'media',
                              model_name = model_name,
                              )

Number of urls collected for vine: 1
Number of urls collected for grass: 28
Number of urls collected for ground: 23
Dataframe created successfully with shape : (52, 4)
Preprocess_input function for 'ResNet50' loaded successfully.


# 5-  Evaluate the model on the test dataset

#### Now we can evaluate the model on the test dataset
#### The metrics displayed here are the ones that have been set while compiling the model before training

In [16]:
# Evaluate the model
results = loaded_model.evaluate(test_dataset)

print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 232ms/step - accuracy: 0.0520 - loss: 10.9764 - precision: 0.0520 - recall: 0.0520
Test Loss: 12.354569435119629
Test Accuracy: 0.01923076994717121
Test Precision: 0.01923076994717121
Test Recall: 0.01923076994717121
